In [25]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from input_pipeline import dataset_pipeline, get_param_dict
from tqdm import tqdm
import time

In [26]:
trainset_path = "../GetData/Rust/get_data_rust/Data/Trainset"
testset_path = "../GetData/Rust/get_data_rust/Data/Testset"
batch_size = 20
train_set = dataset_pipeline(trainset_path, flatten=False, batch_size=batch_size)
train_params = get_param_dict(trainset_path)
test_set = dataset_pipeline(testset_path, flatten=False, batch_size=batch_size)
test_params = get_param_dict(testset_path)
train_set = train_set.map(lambda x: (x+2)/2)
test_set = test_set.map(lambda x: (x+2)/2)
print("Trainset parameters: " + str(train_params))
print("Testset parameters: " + str(test_params))

Getting data from ../GetData/Rust/get_data_rust/Data/Trainset
Got 10000 samples
Getting data from ../GetData/Rust/get_data_rust/Data/Testset
Got 1000 samples
Trainset parameters: {'Steps': '1000', 'Simulatiton Number': '10000', 'Temperature': '2.73', 'Magnetic Field': '0', 'Mattize Size': '64'}
Testset parameters: {'Steps': '1000', 'Simulatiton Number': '1000', 'Temperature': '2.73', 'Magnetic Field': '0', 'Mattize Size': '64'}


In [27]:
class VAE(tf.keras.Model):
  """Variational autoencoder."""

  def __init__(self, latent_dim, mattize_size, inner_filters=4, outter_filters=2):
    super(VAE, self).__init__()
    self.latent_dim = latent_dim
    assert mattize_size%4 == 0
    dense_units = mattize_size // 4
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(mattize_size, mattize_size, 1)),
            tf.keras.layers.Conv2D(
                filters=outter_filters, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=inner_filters, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=dense_units*dense_units*inner_filters, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(dense_units, dense_units, inner_filters)),
            tf.keras.layers.Conv2DTranspose(
                filters=inner_filters, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=outter_filters, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [28]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [29]:
optimizer = tf.keras.optimizers.Adam(1e-4)
epochs = 5
latent_dim = 16
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])
model = VAE(latent_dim, 64)

In [30]:
def generate_and_save_images(model, epoch, test_sample):
  mean, logvar = model.encode(test_sample)
  z = model.reparameterize(mean, logvar)
  predictions = model.sample(z)
  fig = plt.figure(figsize=(4, 4))
  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i + 1)
    plt.imshow(predictions[i, :, :, 0], cmap='gray')
    plt.axis('off')

  # tight_layout minimizes the overlap between 2 sub-plots
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [31]:
# Pick a sample of the test set for generating output images
assert batch_size >= num_examples_to_generate
for test_batch in test_set.take(1):
  test_sample = test_batch[0:num_examples_to_generate, :, :, :]

In [32]:
for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in tqdm(train_set):
    train_step(model, train_x, optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_set:
    loss(compute_loss(model, test_x))
  elbo = -loss.result()
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))

100%|██████████| 500/500 [00:10<00:00, 46.48it/s]


Epoch: 1, Test set ELBO: 2620.1708984375, time elapse for current epoch: 10.760489463806152


100%|██████████| 500/500 [00:08<00:00, 56.53it/s]


Epoch: 2, Test set ELBO: 9324422.0, time elapse for current epoch: 8.848400115966797


100%|██████████| 500/500 [00:09<00:00, 53.00it/s]


Epoch: 3, Test set ELBO: 191025248.0, time elapse for current epoch: 9.436461925506592


100%|██████████| 500/500 [00:08<00:00, 57.24it/s]


Epoch: 4, Test set ELBO: 1194372608.0, time elapse for current epoch: 8.737842321395874


100%|██████████| 500/500 [00:08<00:00, 55.73it/s]


Epoch: 5, Test set ELBO: nan, time elapse for current epoch: 8.975056409835815
